# Custom vectorizer

In [1]:
import os
os.chdir('..')
print(os.getcwd())

/home/mono/materias/pln/repo


In [2]:
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pickle

In [3]:
from sentiment.tass import InterTASSReader

# Train data
reader = InterTASSReader('corpus/InterTASS/PE/intertass-PE-train-tagged.xml')
tweets = list(reader.tweets())  # iterador sobre los tweets
X = list(reader.X())  # iterador sobre los contenidos de los tweets
y = list(reader.y())  # iterador sobre las polaridades de los tweets

# Development data
reader = InterTASSReader('corpus/InterTASS/PE/intertass-PE-development-tagged.xml')
tweets = list(reader.tweets())  # iterador sobre los tweets
X_dev = list(reader.X())  # iterador sobre los contenidos de los tweets
y_dev = list(reader.y())  # iterador sobre las polaridades de los tweets

In [11]:
# find tweet with string
for i, x in enumerate(X):
    if 'no' in x:
        break

print(i)
print(X[i])

0
Sin ser fan de Juan Gabriel, siempre supe que era una fuerza de la naturaleza. Hoy escuché "Querida", y me dio una ternura enorme.


### CountVectorizer as is

In [30]:
vect = CountVectorizer()
print(vect.fit_transform(X))


  (0, 1408)	1
  (0, 4049)	1
  (0, 1204)	1
  (0, 2551)	1
  (0, 3412)	1
  (0, 1482)	1
  (0, 1967)	1
  (0, 2760)	1
  (0, 2237)	1
  (0, 1735)	1
  (0, 4194)	2
  (0, 1452)	1
  (0, 3386)	1
  (0, 3943)	1
  (0, 3783)	1
  (0, 1754)	1
  (0, 2183)	1
  (0, 1046)	2
  (0, 1616)	1
  (0, 3750)	1
  (0, 3812)	1
  (1, 172)	1
  (1, 4295)	1
  (1, 4371)	1
  (1, 4084)	1
  :	:
  (997, 2855)	1
  (997, 4042)	1
  (997, 2926)	1
  (997, 867)	1
  (997, 2399)	1
  (997, 4089)	1
  (997, 657)	1
  (997, 818)	1
  (997, 2551)	1
  (997, 3386)	1
  (998, 1790)	1
  (998, 1788)	1
  (998, 4210)	1
  (998, 2774)	1
  (998, 1435)	1
  (998, 175)	1
  (998, 2237)	1
  (999, 968)	1
  (999, 1295)	1
  (999, 3727)	1
  (999, 3493)	1
  (999, 2388)	1
  (999, 2551)	2
  (999, 3386)	2
  (999, 1046)	1


## Changing tokenizer

In [20]:
vect = CountVectorizer(tokenizer=word_tokenize)
vectorize = vect.build_analyzer()
print(vectorize(X[5]))

['@', 'fiorela_gue', 'aw', '!', 'recién', 'veo', 'esto', ',', 'sorrry', '!', 'tu', 'pues', 'me', 'dices', 'algo', 'y', 'luego', 'queda', 'en', 'nada', 'vamos', 'hoy', 'a', 'la', 'reu', 'de', 'mi', 'prima', '?', 'le', 'dije', 'a', 'mari']


In [20]:
def neg_handling_tokenizer(s, max_negations=3):
    neg_words = ['no', 'tampoco', 'ni']
    tokens = word_tokenize(s)
    make_opposite = False
    neg_count = 0
    for i in range(len(tokens)):
        if re.search(r'[.?\-",]+', tokens[i]):
            make_opposite = False
        if make_opposite and \
            tokens[i] not in neg_words and \
            neg_count <= max_negations:

            tokens[i] = 'NOT_' + tokens[i]
            neg_count += 1
        if tokens[i] in neg_words:
            make_opposite = True
            neg_count = 0
    return tokens

def build_tokenizer(max_negations=2):
    return lambda s: neg_handling_tokenizer(s, max_negations)

vect = CountVectorizer(tokenizer=build_tokenizer())
vectorize = vect.build_analyzer()
print(vectorize(X[10]))

['+saliste', 'ayer', '?', '-no', ',', 'me', 'quedé', 'en', 'casa', '.', '+qué', 'aburrido', 'te', 'has', 'vuelto', '!', '-eso', 'no', 'NOT_dices', 'NOT_cuando', 'NOT_llega', 'la', 'quincena', 'y', 'no', 'NOT_tienes', 'ni', 'NOT_para', 'NOT_el', 'NOT_pasaje', '.']


## Changing preprocessor

In [32]:
def my_preprocessor(doc):
    tagged_users_pat = r'@\w+'
    urls_pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    vocals_rep_pat = r'(\w)\1{1,}'
    laugh_pat = r'\b[jah]+\b'
    return re.sub(tagged_users_pat, '',
           re.sub(laugh_pat, '<LAUGH>',
           re.sub(urls_pat, '',
           re.sub(vocals_rep_pat, '\g<1>',
           doc.lower()))))

vect = CountVectorizer(preprocessor=my_preprocessor)
vectorize = vect.build_analyzer()
print(vectorize(X[1]))

['ayer', 'preguntaban', 'dónde', 'están', 'las', 'solteras', 'todo', 'mi', 'grupo', 'alza', 'la', 'mano', 'yo', 'la', 'única', 'que', 'no', 'todas', 'voltean', 'LAUGH', 'verme', 'LAUGH']


## Grid-search params finding

In [72]:
vect = CountVectorizer()
pipe = Pipeline([
    ('vect', vect),
     ('clf', SVC(gamma='auto', probability=True))
])

train_X = [
    'el gato come pescado',
    'la gata come salmon',
    'el gato come salmon'
]
train_Y = [1, 0, 1]

param_dict = {
    'clf__C': [1, 10, 100]
}

cv = GridSearchCV(pipe, param_dict, cv=[[[0, 1], [2]]])
cv.fit(train_X, train_Y)
print(cv.predict_proba(['la gata come', 'la gata come salmon']))

[[0.24483943 0.75516057]
 [0.20822938 0.79177062]]


## Analyzing the models

In [7]:
with open('models/ES-maxent.model', 'rb') as fp:
    model = pickle.load(fp)
        
y_pred = model.predict(X_dev)

from sentiment.analysis import print_maxent_features, print_feature_weights_for_item

vect = model._pipeline.named_steps['vect']
clf = model._pipeline.named_steps['clf']
print_maxent_features(vect, clf)

N:
	! gracias buena : mejor ([-0.99926348 -0.54140291 -0.50538172 -0.49823128 -0.48591543])
	mismo odio ni triste no ([0.47739055 0.60890999 0.65199539 0.80367345 0.97239018])
NEU:
	# gracias hoy ! ? ([-0.55027125 -0.54565408 -0.44861801 -0.39353667 -0.38358295])
	NOT_pasa vez casa aunque sido ([0.33468925 0.35213721 0.37396061 0.3906679  0.410102  ])
NONE:
	no ... mal hoy ser ([-0.70112823 -0.49297417 -0.42632923 -0.34505574 -0.34179787])
	vídeo jugar alguna semana ? ([0.39499268 0.39759467 0.39960258 0.45413998 0.99736069])
P:
	no triste ? ni odio ([-0.71764461 -0.50168295 -0.42048146 -0.39279131 -0.32097587])
	mejor genial buen gracias ! ([0.63658058 0.6450265  0.85123413 0.85493357 1.14742911])


In [24]:
wrongly_tagged = [(X_dev[i], y_dev[i], y_pred[i]) 
                  for i in range(len(X_dev)) if y_pred[i] != y_dev[i]]
sorted(wrongly_tagged, key=lambda t: -len(t[0]))


[('@Sol_4843 si lo es! pero a mi no me pegó del todo la sentí bien poltergeist, lo cual no quiere decir que sea mala solo que no me "atrapó"',
  'NEU',
  'P'),
 ('Buenos días!!!! mi pensamiento esta mañana al despertar es que si o si tengo que ir a dar un paseo al parque y disfrutar de la naturaleza',
  'NONE',
  'P'),
 ('Vine en bici a un curso de mi chamba en San Isidro. Genial...!! Solo mi pelo ha sufrido estragos. Pensare en algo para q mañana no suceda',
  'NONE',
  'P'),
 ('@adricordva pero y que pasa con "todo lo puedo en cristo que me fortalece" "solo Dios le da batallas a los fuertes" nada de eso? jajajaja',
  'N',
  'NONE'),
 ('En 1990 una amiga quiso ser policía. Sus padres se opusieron xq las mujeres no podían ser oficiales. Luego de 26 años, tenemos GENERALAS.',
  'NONE',
  'N'),
 ('@elmejorerico @silva_alvan @meiermq si los Ramirez dicen q son honrados,becerril q es buen padre,Chacón q sí tiene educ.X q no me crees ?',
  'NONE',
  'N'),
 ('@jakahatt algún dia se mejorarán